In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_9월_전처리.txt", encoding='UTF-8')
print(df.shape, df.columns)
corpus = df['split_str'].to_list() # 문장(X문서X)으로 가져와서 실행.
# ↑ 추후에 다른 분석 시도할 때 문장 별 형태소가 필요할 때도 있음. 문장별 형태소와 이를 문서로 합친 게 있으면 편하기 때문에 문장별로 형태소 분석 실행.
print(len(corpus))

(1021057, 2) Index(['split_str', 'org_idx'], dtype='object')
1021057


In [3]:
''' 형태소 분석 '''
nouns = []
verbs = []
adjs = []
josas = [] # 불용어 목록에 조사 사용하기 위함.
morphs = [] # 품사 태깅 없이 형태소만 추출. 추후 TF-IDF, N-gram등 텍스트 분석은 이 데이터로 진행함.
pos_list = [] # 감성분석할 때 필요.

In [4]:
for i in tqdm(range(len(corpus))):
    # 답변에서 형태소/품사 추출
    try:
        a = okt.pos(corpus[i], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
        m = okt.morphs(corpus[i], norm=True, stem=False)
        morphs.append(m) # 형태소 추가
        pos_list.append(a) # 형태소/품사 추가
        for x, y in a:
            # 품사가 명사면 명사 리스트에 단어 추가
            if y == 'Noun':
                nouns.append(x)
            # 품사가 동사면 동사 리스트에 단어 추가
            elif y == 'Verb':
                verbs.append(x)
            # 품사가 형용사면 형용사 리스트에 단어 추가
            elif y == 'Adjective':
                adjs.append(x)
		    # 품사가 조사면 조사 리스트에 단어 추가.
            elif y == 'Josa':
                josas.append(x)
    except:
        pass

100%|██████████| 1021057/1021057 [18:50:12<00:00, 15.06it/s]   


In [5]:
# 명사, 동사, 형용사 리스트에서 10개 단어씩 출력해서 확인
print(nouns[0:10])
print(verbs[0:10])
print(adjs[0:10])
print(josas[0:10])

['송천', '도서관', '전라북도', '전주시', '덕진구', '솔', '로', '송천', '도서관', '문화']
['하다', '받다', '하다', '하다', '하다', '인정받다', '차다', '새다', '만들다', '인정받다']
['있다', '불편하다', '불편하다', '불편하다', '불편하다', '있다', '넓다', '있다', '중요하다', '있다']
['에서', '가', '와의', '가', '는', '이다', '는', '과', '로', '의']


In [6]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(nouns))) # 이 값을 밑에 most_common 안에 넣으면 됨.
noun_cnt = Counter(nouns).most_common()

명사 = []
명사빈도 = []
for a, b in noun_cnt:
    명사.append(a)
    명사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
noun_df = pd.DataFrame({'명사':명사, '명사빈도':명사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
noun_df.to_csv("../dataset/2022/본문_형태소분석/9월/나무문화_9월_형태소분석(명사).csv", index=False, encoding='UTF-8')
noun_df

163845


,명사,명사빈도
0,것,224338
1,수,135964
2,그,115769
3,이,110900
4,곳,82028
...,...,...
163840,글레라,1
163841,페테,1
163842,레갈,1
163843,뮈스캇,1


In [7]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(verbs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
verb_cnt = Counter(verbs).most_common()

동사 = []
동사빈도 = []
for a, b in verb_cnt:
    동사.append(a)
    동사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
verb_df = pd.DataFrame({'동사':동사, '동사빈도':동사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
verb_df.to_csv("../dataset/2022/본문_형태소분석/9월/나무문화_9월_형태소분석(동사).csv", index=False, encoding='UTF-8')
verb_df

1887


,동사,동사빈도
0,하다,950343
1,되다,193977
2,보다,113265
3,않다,91704
4,되어다,85846
...,...,...
1882,자각시키다,1
1883,촐랑대다,1
1884,고꾸라뜨리다,1
1885,아슬랑거리다,1


In [8]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(adjs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
adj_cnt = Counter(adjs).most_common()

형용사 = []
형용사빈도 = []
for a, b in adj_cnt:
    형용사.append(a)
    형용사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
adj_df = pd.DataFrame({'형용사':형용사, '형용사빈도':형용사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
adj_df.to_csv("../dataset/2022/본문_형태소분석/9월/나무문화_9월_형태소분석(형용사).csv", index=False, encoding='UTF-8')
adj_df

1876


,형용사,형용사빈도
0,있다,455447
1,없다,85003
2,좋다,75777
3,같다,71433
4,이다,60334
...,...,...
1871,쾌할하다,1
1872,서글하다,1
1873,도렷하다,1
1874,민완하다,1


In [9]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(josas))) # 이 값을 밑에 most_common 안에 넣으면 됨.
josa_cnt = Counter(josas).most_common()

조사 = []
조사빈도 = []
for a, b in josa_cnt:
    조사.append(a)
    조사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
josa_df = pd.DataFrame({'조사':조사, '조사빈도':조사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
josa_df.to_csv("../dataset/2022/본문_형태소분석/9월/나무문화_9월_형태소분석(조사).csv", index=False, encoding='UTF-8')
josa_df

391


,조사,조사빈도
0,을,775946
1,의,632526
2,이,595346
3,에,572955
4,를,378740
...,...,...
386,으로만은,1
387,에게까지는,1
388,조차를,1
389,같이만,1


In [10]:
df_concat = pd.concat([noun_df,verb_df,adj_df],axis=1)
df_concat.to_csv("../dataset/2022/본문_형태소분석/나무문화_9월_단어종합.txt", index=False, encoding='UTF-8')

In [11]:
df["morphs"] = morphs
df["pos_tag"] = pos_list
df

,split_str,org_idx,morphs,pos_tag
0,50m NAVER Corp 송천도서관 전라북도 전주시 덕진구 솔내로 212 송천도서...,0,"[50, m, NAVER, Corp, 송천, 도서관, 전라북도, 전주시, 덕진구, ...","[(50, Number), (m, Alpha), (NAVER, Alpha), (Co..."
1,강사는 불편한편의점 의 김호연 작가이다 불편한 편의점 2 저자 김호연 출판 나무옆의...,0,"[강사, 는, 불편한, 편의점, 의, 김호연, 작가, 이다, 불편한, 편의점, 2,...","[(강사, Noun), (는, Josa), (불편하다, Adjective), (편의..."
2,주제 소설가의 삶과 문화콘텐츠 스토리텔링 강의 시작전 22년간 영화 만화 출판 관련...,0,"[주제, 소설가, 의, 삶, 과, 문화, 콘텐츠, 스토리텔링, 강의, 시, 작전, ...","[(주제, Noun), (소설가, Noun), (의, Josa), (삶, Noun)..."
3,문화콘텐츠 문화콘텐츠란 인간의 감성 창의성 상상력을 원천으로 한 문화상품 문화콘텐츠...,0,"[문화, 콘텐츠, 문화, 콘텐츠, 란, 인간, 의, 감성, 창의성, 상상력, 을, ...","[(문화, Noun), (콘텐츠, Noun), (문화, Noun), (콘텐츠, No..."
4,영화 드라마 만화 대중음악 뮤지컬 모바일게임 인터넷 모바일 테마파크 전시 축제 공간...,0,"[영화, 드라마, 만화, 대중음악, 뮤지컬, 모바일, 게임, 인터넷, 모바일, 테마...","[(영화, Noun), (드라마, Noun), (만화, Noun), (대중음악, N..."
...,...,...,...,...
1021052,라벨에 BIO라는 단어가 지워져 있습니다,20813,"[라벨, 에, BIO, 라는, 단어, 가, 지워져, 있습니다]","[(라벨, Noun), (에, Josa), (BIO, Alpha), (라는, Jos..."
1021053,원재료의 일부로 들어간 유기농 설탕과 시나몬이 유럽에서는 유기농 인증서가 있음에도 ...,20813,"[원, 재료, 의, 일부, 로, 들어간, 유기농, 설탕, 과, 시나몬, 이, 유럽,...","[(원, Modifier), (재료, Noun), (의, Josa), (일부, No..."
1021054,아버지를 아버지라 부르지 못하는 홍길동처럼 유기농을 유기농이라 부르지 못하는 홍길동...,20813,"[아버지, 를, 아버지, 라, 부르지, 못, 하는, 홍길동, 처럼, 유기농, 을, ...","[(아버지, Noun), (를, Josa), (아버지, Noun), (라, Josa..."
1021055,무더운 여름 건강 조심하십시요,20813,"[무더운, 여름, 건강, 조심하십시요]","[(무덥다, Adjective), (여름, Noun), (건강, Noun), (조심..."


In [12]:
df.to_csv("../dataset/2022/본문_형태소분석/나무문화_본문_9월_형태소분석.txt", index=False, encoding='UTF-8')